## Imports

In [1]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets

from utils import combine_strings_with_whitespace

In [2]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [3]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [4]:
from templates import DatasetTemplates

In [5]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

# Load the dataset

In [6]:
VERSION = "v2"

In [7]:
!ls -lah datasets | grep {VERSION} | grep imdb
# !ls -lah datasets | grep {VERSION}

drwxr-xr-x  3 augustas Domain Users  25K Jul  4 23:02 burns_datasets_VINC_imdb_ppo_training_raw_v2
drwxr-xr-x  3 augustas Domain Users  25K Jul  4 23:02 burns_datasets_VINC_imdb_train_raw_v2
-rw-r--r--  1 augustas Domain Users 7.4M Jul  4 23:03 burns_datasets_VINC_imdb_train_v2.parquet
-rw-r--r--  1 augustas Domain Users 1.2M Jul  4 23:06 burns_datasets_VINC_imdb_validation_v2.parquet


In [8]:
# Set dataset_dict to test_dataset
dataset_dict = datasets.DatasetDict.load_from_disk(
    # f"datasets/burns_datasets_VINC_ppo_training_raw_{VERSION}"
    f"datasets/burns_datasets_VINC_imdb_ppo_training_raw_{VERSION}"
)
dataset_dict

DatasetDict({
    imdb: Dataset({
        features: ['text', 'label'],
        num_rows: 15000
    })
})

In [9]:
sum(dataset.num_rows for dataset in dataset_dict.values())

15000

In [10]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

imdb: 2


## Load the templates

In [11]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [12]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

imdb: 13


In [13]:
# dataset_template_dict["ag_news"].templates

In [14]:
for dataset_name in dataset_template_dict:
    good_templates = {
        name: x for name, x in dataset_template_dict[dataset_name].templates.items() if x.metadata.choices_in_prompt
    }
    dataset_template_dict[dataset_name].templates = good_templates
    print(f"{dataset_name}: {len(good_templates)}")

imdb: 4


In [15]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

imdb: 4


In [16]:
%%time

# Reproducibility
random.seed(2023)

ALLOWED_KEYS = ["prompt", "best_response", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        q, a = template.apply(new_entry)
        new_entry["prompt"] = combine_strings_with_whitespace(a, q)
        if not new_entry["prompt"][-1].isspace():
            new_entry["prompt"] += " "

        # We can now change the label to whether the sample is truthful or not
        new_entry["best_response"] = a.strip()

        # Remove all other keys
        new_entry = { k: v for k, v in new_entry.items() if k in ALLOWED_KEYS }

        # Append to the new dataset
        new_dataset.append(new_entry)

imdb


CPU times: user 25.4 s, sys: 154 ms, total: 25.5 s
Wall time: 25.5 s


In [17]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
    num_rows: 15000
})

In [18]:
current_idx = 0
my_dataset[current_idx]

{'original_dataset': 'imdb',
 'template_name': 'burns_1',
 'prompt': "Consider the following example:  ''' Hated it. If you believe that everyone in the South is dumb, morally bankrupt, stupid, violent, a religious nut, or a child molester, then this film may be for you. Everyone is poor and seemingly ignorant. In one scene, two older men are talking in a general store and one mentions that he had molested a set of sisters before they could tie their shoes. The man seemed proud of his actions, and the other man clearly took it as a normal part of life. Very nice. A teenage girl walks the back roads looking for her sister and no one offers to help her -- despite an obvious limp and lack of food or water (no backpack, etc.). Strathairn's character is not only thoroughly disgusting and slimy, but he is shown to be a religious believer who (typical for Hollywood) reflects the vile nature of Christians. A scene in the movie is highly reminiscent of the end of Cape Fear (the one with DeNiro)

In [19]:
print(f'\'{my_dataset[current_idx]["prompt"]}\'')

'Consider the following example:  ''' Hated it. If you believe that everyone in the South is dumb, morally bankrupt, stupid, violent, a religious nut, or a child molester, then this film may be for you. Everyone is poor and seemingly ignorant. In one scene, two older men are talking in a general store and one mentions that he had molested a set of sisters before they could tie their shoes. The man seemed proud of his actions, and the other man clearly took it as a normal part of life. Very nice. A teenage girl walks the back roads looking for her sister and no one offers to help her -- despite an obvious limp and lack of food or water (no backpack, etc.). Strathairn's character is not only thoroughly disgusting and slimy, but he is shown to be a religious believer who (typical for Hollywood) reflects the vile nature of Christians. A scene in the movie is highly reminiscent of the end of Cape Fear (the one with DeNiro) -- Bible verses being spouted by the bad guy. I am from the Great No

In [20]:
Counter(my_dataset["original_dataset"])

Counter({'imdb': 15000})

In [24]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_imdb_ppo_training_{VERSION}.parquet")
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_ppo_training_{VERSION}.parquet")

In [25]:
!ls -lah datasets | grep {VERSION} | grep imdb
# !ls -lah datasets | grep {VERSION}

drwxr-xr-x  3 augustas Domain Users  25K Jul  4 23:02 burns_datasets_VINC_imdb_ppo_training_raw_v2
-rw-r--r--  1 augustas Domain Users  12M Jul  4 23:07 burns_datasets_VINC_imdb_ppo_training_v2.parquet
drwxr-xr-x  3 augustas Domain Users  25K Jul  4 23:02 burns_datasets_VINC_imdb_train_raw_v2
-rw-r--r--  1 augustas Domain Users 7.4M Jul  4 23:03 burns_datasets_VINC_imdb_train_v2.parquet
-rw-r--r--  1 augustas Domain Users 1.2M Jul  4 23:06 burns_datasets_VINC_imdb_validation_v2.parquet
